In [1]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

/Users/connorparish/miniconda3/envs/hindsight_server/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model_id = "meta-llama/Llama-3.2-11B-Vision"
model_id = "neuralmagic/Llama-3.2-11B-Vision-Instruct-FP8-dynamic"

In [3]:
model = MllamaForConditionalGeneration.from_pretrained(
    model_id
).to("cpu")

Loading checkpoint shards: 100%|██████████| 3/3 [00:26<00:00,  8.71s/it]
Some weights of the model checkpoint at neuralmagic/Llama-3.2-11B-Vision-Instruct-FP8-dynamic were not used when initializing MllamaForConditionalGeneration: ['language_model.model.layers.0.mlp.down_proj.weight_scale', 'language_model.model.layers.0.mlp.gate_proj.weight_scale', 'language_model.model.layers.0.mlp.up_proj.weight_scale', 'language_model.model.layers.0.self_attn.k_proj.weight_scale', 'language_model.model.layers.0.self_attn.o_proj.weight_scale', 'language_model.model.layers.0.self_attn.q_proj.weight_scale', 'language_model.model.layers.0.self_attn.v_proj.weight_scale', 'language_model.model.layers.1.mlp.down_proj.weight_scale', 'language_model.model.layers.1.mlp.gate_proj.weight_scale', 'language_model.model.layers.1.mlp.up_proj.weight_scale', 'language_model.model.layers.1.self_attn.k_proj.weight_scale', 'language_model.model.layers.1.self_attn.o_proj.weight_scale', 'language_model.model.layers.1.sel

In [4]:
processor = AutoProcessor.from_pretrained(model_id)

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "If I had to write a haiku for this one, it would be: "}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, return_tensors="pt").to(model.device)

In [5]:
output = model.generate(**inputs, max_new_tokens=30)

In [ ]:
print(processor.decode(output[0]))